#Permission-setup Data generation for UC demo notebook

In [0]:
catalog = "uc_acl_learning"
schema = dbName = db = "dev"

In [0]:
%run ./00-global-setup-v2

In [0]:
DBDemos.setup_schema(catalog, db, False)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS analyst_permissions (
  analyst_email STRING,
  country_filter STRING,
  gdpr_filter LONG); 

-- ALTER TABLE uc_acl.users OWNER TO `account users`;
-- ALTER TABLE analyst_permissions OWNER TO `account users`;
-- GRANT SELECT, MODIFY on TABLE analyst_permissions TO `account users`;

CREATE TABLE IF NOT EXISTS customers (
  id STRING,
  creation_date STRING,
  firstname STRING,
  lastname STRING,
  country STRING,
  email STRING,
  address STRING,
  gender DOUBLE,
  age_group DOUBLE); 
-- ALTER TABLE customers OWNER TO `account users`; -- for the demo only, allow all users to edit the table - don't do that in production!
-- GRANT SELECT, MODIFY on TABLE customers TO `account users`;

In [0]:
from pyspark.sql.functions import col
import pandas as pd

df = pd.read_parquet("https://dbdemos-dataset.s3.amazonaws.com/retail/c360/users_parquet/users.parquet.snappy")

spark.createDataFrame(df).withColumn('age_group', col("age_group").cast("double")) \
                         .withColumn('gender', col("gender").cast("double")) \
                         .write.mode('overwrite').option('mergeSchema', 'true').saveAsTable("customers")

In [0]:
import random

countries = ['FR', 'USA', 'SPAIN']
current_user = spark.sql('select current_user() as user').collect()[0]["user"]
workspace_users = df['email'][-30:].to_list() + [current_user]

user_data = [(u, countries[random.randint(0,2)], random.randint(0,1)) for u in workspace_users]

spark.createDataFrame(user_data, ['analyst_email', 'country_filter',"gdpr_filter"]) \
       .repartition(3).write.mode('overwrite').saveAsTable("analyst_permissions")